In [21]:
import numpy as np
import matplotlib.pyplot as plt
from nd2reader import ND2Reader
from skimage.exposure import rescale_intensity
from cellpose import models
from skimage.segmentation import mark_boundaries
from skimage.filters import gaussian
from skimage.morphology import disk,white_tophat,disk,remove_small_objects
from skimage.measure import label,regionprops_table
import pandas as pd
import glob
import os

# Define functions

In [269]:
#Used to find max intensity projection of ND2 stack for cytoplasm and nuclei channels
def nd2_max_projection(nd2_object):
    #Get frame dimensions
    z_dim = nd2_object.sizes['z']
    x_dim = nd2_object.frame_shape[0]
    y_dim = nd2_object.frame_shape[1]
    stack = np.empty([z_dim,x_dim,y_dim])

    #Import all cytoplasm focal planes
    nd2_object.default_coords['c'] = 0 #Cyto Channel
    for z in range(z_dim):
        stack[z,:,:] = nd2_object[z]

    #Calculate maximum intensity projects for cyto channel
    max_intensity_cyto = np.amax(stack,axis=(0))

    #Import all nuclei focal planes
    nd2_object.default_coords['c'] = 1 #Nuclei Channel
    for z in range(z_dim):
        stack[z,:,:] = nd2_object[z]

    #Calculate maximum intensity projects for nuclei channel
    max_intensity_nuclei = np.amax(stack,axis=(0))

    return max_intensity_cyto,max_intensity_nuclei

#Used to import ND2 image stacks and return max, intensity projection of cytoplasm and nuclei channels
def import_stack(img_path,focal_plane_cyto=0,focal_plane_nuclei=0,min_percentile_cyto = 0,max_percentile_cyto = 100,min_percentile_nuclei = 0,max_percentile_nuclei = 100,max_intensity_projection = None):

  #Create nd2 image object
  nd2_object = ND2Reader(img_path)

  #If we want maximum projection images
  if max_intensity_projection == True:
    channel_cyto,channel_nuclei = nd2_max_projection (nd2_object)

  #If we want any focal plane
  else:
    #Import cytoplasm image at desired focal plane
    nd2_object.default_coords['c'] = 0
    channel_cyto = nd2_object[focal_plane_cyto]

    #Import nuclei image at desired focal plane
    nd2_object.default_coords['c'] = 1
    channel_nuclei = nd2_object[focal_plane_nuclei]

    nd2_object.close()


  channel_cyto_no_saturation =  channel_cyto

  #rescale cyto channel based on max and min cyto percentile limits
  v_min, v_max = np.percentile(channel_cyto, (min_percentile_cyto, max_percentile_cyto))
  channel_cyto = rescale_intensity(channel_cyto, in_range=(v_min, v_max))

  channel_nuclei_no_saturation = channel_nuclei

  #rescale nuclei channel based on max and min nuclei percentile limits
  v_min, v_max = np.percentile(channel_nuclei, (min_percentile_nuclei, max_percentile_nuclei))
  channel_nuclei = rescale_intensity(channel_nuclei, in_range=(v_min, v_max))

  #Create 3d array with with channel 0: cytoplasm, channel 1: nuclei
  output_stack = np.zeros([3,channel_cyto.shape[0],channel_cyto.shape[1]])
  output_stack[1,:,:] = channel_cyto
  output_stack[2,:,:] = channel_nuclei

  #Create 3d array with with channel 0: cytoplasm, channel 1: nuclei
  output_stack_no_saturation = np.zeros([3,channel_cyto.shape[0],channel_cyto.shape[1]])
  output_stack_no_saturation[1,:,:] = channel_cyto_no_saturation
  output_stack_no_saturation[2,:,:] = channel_nuclei_no_saturation

  #Convert to float32 for cellpose
  output_stack = (output_stack).astype(dtype = 'float32') 
  output_stack_no_saturation = (output_stack_no_saturation/65535.0).astype(dtype = 'float32')

  return output_stack,output_stack_no_saturation


#Segments the nucleus. Pass the 3D image stack and choose manual segmentation threshold for nuclei
def nuclei_segmentation(stack,thresh_nuclei=0.06,show_results = None):
    #Blur nuclei image
    nuclei_image = gaussian(stack[2,:,:],1)

    mask_nuclei = ((nuclei_image > thresh_nuclei)*1).astype('float32')

    if show_results == True:
        plt.figure(figsize = (20,5))
        plt.subplot(1,2,1)
        plt.imshow(stack[2,:,:])
        plt.title('Nuclei Channel')
        plt.colorbar()

        plt.subplot(1,2,2)
        plt.imshow(mask_nuclei)
        plt.title('Nuclei Channel')
    return mask_nuclei


#Function to segment the cells using Cellpose
def cell_segmentation(stack,diameter = 200,flow_threshold = 0,use_gpu = False,show_results = False):
    model = models.Cellpose(gpu=use_gpu, model_type='cyto',net_avg=False)
    mask_cyto, _, _, _ = model.eval(stack, diameter=diameter, flow_threshold=flow_threshold, channels=[2,3])

    if show_results == True:
        plt.figure(figsize = (15,15))
        segmentation_results = mark_boundaries(stack[1,:,:],mask_cyto)
        plt.imshow(segmentation_results)

    return mask_cyto


#Function to segment LC3 dots using white tophot and manual thresholding
def LC3_segmentation(stack,mask,threshold,tophat_size = 5,minimum_dot_size = 0,display_results = False):
    #image: raw max_intensity projection of cellular channel as input image (No image saturation works best)
    #mask: cytoplasm mask (no nuclei)
    #threshold: value to segment the proteins
    #plot_results: if true, will show images of steps
    # minimum_dot_size: removes leftover dots below this threshold area
    
    #Use cyto channel
    image = stack[1,:,:]

    #Tophat morphological filter to supress large objects in image
    LC3_tophat = white_tophat(image,disk(tophat_size))

    #Manual Thresholding
    LC3_segmented = LC3_tophat > threshold   

    #Remove nuclei + out-of-cell dots
    LC3_segmented = LC3_segmented * mask

    #Removes dots below specified area if needed
    if minimum_dot_size > 0 :
        LC3_segmented = remove_small_objects(np.bool_(LC3_segmented),min_size = 3)

    #Show overlayed image if needed
    if display_results == True:
        
        segmentation_results = mark_boundaries(stack[1,:,:],np.bool_(LC3_segmented))
        plt.figure(figsize= (10,10))
        plt.imshow(segmentation_results)         
    return np.uint16(LC3_segmented)

## Run Program
#### Main cell to loop through folder with images and return .xlsx file with number of proteins detected in each frame and .tiff frames of results

In [335]:
#Enter path of folder containing the images
filepath = '/Users/robertwelch/Desktop/SciLifeLab Code/Chiara Project/Images/oneImage'

pathND2 = filepath +'/*.nd2'
for filename in glob.glob(pathND2):
    print('Analyzing:',filename)

    #Open Max Intensity Projection Image stack with formatting for cellpose
    #stack_with_sat has cytoplasm channel clipped to 95% max intensity, stack_without_sat does not
    stack_with_sat,stack_without_sat = import_stack(filename,min_percentile_cyto=0,max_percentile_cyto=95,min_percentile_nuclei=0,max_percentile_nuclei=100,max_intensity_projection=True)

    # Create Nuclei Mask
    mask_nuclei = nuclei_segmentation(stack_with_sat,thresh_nuclei=0.06,show_results=False)

    #Create Cell Mask
    mask_cyto = cell_segmentation(stack_with_sat,diameter = 200,flow_threshold=0,use_gpu = False,show_results = False)

    #Create cell mask with no nuclei
    net_mask = ((mask_cyto>0) - mask_nuclei)>0   

    # #Segment the LC3+ dots
    LC3_segmented = LC3_segmentation(stack_with_sat,net_mask,threshold = 0.09,minimum_dot_size=5,display_results=False)
    labeled_LC3 = label(LC3_segmented)

    #Identify with cell LC3+ proteins appear in
    props = regionprops_table(labeled_LC3,mask_cyto,properties=('intensity_min','label'))

    #Convert regionprops results to pandas data frame and count number of LC3+ per cell
    data_holder = pd.DataFrame(props)
    data_holder = data_holder['intensity_min'].value_counts().reset_index()
    
    # Write results
    #data_holder['intensity_min'].to_excel(results_path,index = False)

    #Results header string - uses last 4 numbers in .nd2 file name
    header = ['Cell Label ID : Image {0}'.format(filename[-8:-4]),'LC3+ Count: Image {0}'.format(filename[-8:-4])]
    
    #If First file, write an excel file to store excel results
    new_storage_directory = os.path.join(filepath, filepath+'/Results')
    results_path = new_storage_directory+'/LC3 Counts.xlsx'
    
    #Check if storage directory exits, if not make one
    if os.path.exists(new_storage_directory):
        pass  
    else:
        os.mkdir(new_storage_directory) 

    #create excel results file if it does not exist, otherwise write to existing
    if os.path.exists(results_path) is False:
        data_holder.to_excel(results_path,index = False,header = header)
         
    else:
        #Open Exisisting excel data
        existing_excel = pd.read_excel(results_path)

        #Add results of new image to existing results
        new_excel = pd.concat([existing_excel,data_holder],axis = 1,ignore_index=False)

        #Rename headers of new data
        new_excel = new_excel.rename(columns={'index': header[0],'intensity_min':header[1]})

        #Write new file
        new_excel.to_excel(results_path,index = False)

    #Save Tiff. Image of results
    fig,ax = plt.subplots(1,3)
    fig.set_size_inches(20,20)

    ax[0].imshow(mark_boundaries(stack_with_sat[1,:,:],mask_cyto))
    ax[0].set_title('Cell Segmentation')

    ax[1].imshow(mark_boundaries(stack_without_sat[2,:,:],label(mask_nuclei)))
    ax[1].set_title('Nuclear Segmentation')

    ax[2].imshow(mark_boundaries(stack_with_sat[1,:,:],labeled_LC3))
    ax[2].set_title('LC3 Detected')

    #If you want to speed up significant, reduce dpi (resolution of images or comment out saving image results)
    plt.tight_layout()
    plt.savefig(new_storage_directory+'/results-{0}.tiff'.format(filename[-8:-4]),dpi = 200)
    plt.close('all')


Analyzing: /Users/robertwelch/Desktop/SciLifeLab Code/Chiara Project/Images/oneImage/WellB04_Channel Kinetix Single band EGFP, Kinetix Single  Hoechst_Seq0002.nd2
2022-02-01 08:56:46,778 [INFO] >>>> using CPU
2022-02-01 08:56:46,868 [INFO] ~~~ FINDING MASKS ~~~
2022-02-01 08:56:50,917 [INFO] >>>> TOTAL TIME 4.05 sec
Analyzing: /Users/robertwelch/Desktop/SciLifeLab Code/Chiara Project/Images/oneImage/WellB02_Channel Kinetix Single band EGFP, Kinetix Single  Hoechst_Seq0000.nd2
2022-02-01 08:56:56,129 [INFO] >>>> using CPU
2022-02-01 08:56:56,221 [INFO] ~~~ FINDING MASKS ~~~
2022-02-01 08:57:00,777 [INFO] >>>> TOTAL TIME 4.56 sec
